In [1]:
!pip install --upgrade nbformat
!pip install plotly


[notice] A new release of pip available: 22.2.2 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.2.2 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%cd "C:\Users\user\Documents\UST\BDT\MSBD5005\project\data\"

C:\Users\user\Documents\UST\BDT\MSBD5005\project\data


In [106]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 500)

In [9]:
file = './daily/tweet_verified_221130-230418.csv'
df = pd.read_csv(file)

In [120]:
def TweetColAvgByDate(df_all: pd.DataFrame, col: str) -> pd.DataFrame:
    return df_all[['date', col]].groupby('date').mean()

def topNTweetContentByDate(df_all: pd.DataFrame, col: str, topN: int=5) -> pd.DataFrame:
    return df_all[['date', col, 'renderedContent']].sort_values(['date', col, 'renderedContent'], ascending=False).groupby('date').head(topN)[['date', 'renderedContent']].groupby('date')['renderedContent'].apply('\n'.join).reset_index()
    
def avgColAndTopNContent(df_all: pd.DataFrame, col: str, topN: int=5) -> pd.DataFrame:
    return topNTweetContentByDate(df_all, col, topN).join(TweetColAvgByDate(df_all, col), on='date')

In [141]:
df_view = avgColAndTopNContent(df, 'viewCount')
df_like = avgColAndTopNContent(df, 'likeCount')
df_retweet = avgColAndTopNContent(df, 'retweetCount')
df_count = df.groupby('date').size().reset_index(name='count').merge(avgColAndTopNContent(df, 'viewCount')[['date', 'renderedContent']], on='date')

In [144]:
fig = go.Figure(go.Scatter(
    x = df_count['date'],
    y = df_count['count'],
    hovertemplate =
    '<b>number of tweets</b>: %{y:.0f}'+
    '<br><b>date</b>: %{x}<br>'+
    'top tweets by views: <br>%{text}',
    text = df_count['renderedContent'].apply(lambda x: re.sub('\n', '<br>', x)),
    showlegend = False))

fig.add_trace(go.Scatter(
    x = df_like['date'],
    y = df_like['likeCount'],
    hovertemplate =
    '<b>avg like count</b>: %{y:.0f}'+
    '<br><b>date</b>: %{x}<br>'+
    'top tweets: <br>%{text}',
    text = df_like['renderedContent'].apply(lambda x: re.sub('\n', '<br>', x)),
    showlegend = False))

fig.add_trace(go.Scatter(
    x = df_retweet['date'],
    y = df_retweet['retweetCount'],
    hovertemplate =
    '<b>avg retweet count</b>: %{y:.0f}'+
    '<br><b>date</b>: %{x}<br>'+
    'top tweets: <br>%{text}',
    text = df_like['renderedContent'].apply(lambda x: re.sub('\n', '<br>', x)),
    showlegend = False))

# fig.add_trace(go.Scatter(
#     x = df_view['date'],
#     y = df_view['viewCount'],
#     hovertemplate =
#     '<b>avg view count</b>: %{y:.0f}'+
#     '<br><b>date</b>: %{x}<br>'+
#     'top tweets: <br>%{text}',
#     text = df_view['renderedContent'].apply(lambda x: re.sub('\n', '<br>', x)),
#     showlegend = False))


fig.update_layout(
    hoverlabel_align = 'left',
    title = "Overall trend of no. of tweets/retweets/likes from verified users")

fig.show()